# Сравнение решений задачи классификации узлов сети с использованием различных методов векторных представлений узлов

**Содержание:**


1.   Введение
2.   Основная часть\
    2.1.   Работа с данными\
    2.2.   Векторные представления узлов
          2.2.1     Матричное разложение
          2.2.2     Случайные блуждания
    2.3.   Классификация узлов
          2.3.1     Методы машинного обучения
          2.3.2     Нейронная сеть
3.   Заключение



## **1. Введение**

В данной работе рассматривается решение задачи классификации узлов с применением различных методов векторных представлений узлов. Классификация узлов графа является важным инструментом анализа и обработки графовых данных. Такая задача может встретиться в различных областях, например, в работе с социальными сетями или рекомендательными системами. Она состоит в присвоении меток узлам на графе на основе свойств узлов и взаимосвязей между ними.

Для успешного решения задачи классификации часто используются методы векторных представлений узлов, которые позволяют представить узлы в виде векторов, что может быть использовано при применении алгоритмов машинного обучения.

В исследовании будут применяться различные методы веторных представлений, а также несколько алгоритмов классификации узлов сети. По итогам экспериментального сравнения можно будет сделать выводы о том, какие методы и почему лучше решают поставленную задачу лучше или хуже.

В качестве данных для работы был выбран следующий набор:

Сеть из 100 тысяч пользователей, из которых около 5 тысяч были помечены как высказывающие ненависть пользователи или нет.
Также для каждого пользователя были предоставлены несколько актрибутов, связанных с их активностью в социальной сети. Ребра графа являются ретвитами пользователей, поэтому представленный граф является направленным.

Таким образом, классификация будет заключаться в нахождении пользователей, разжигающих ненависть - юинарная классификация


**Задачи:**


1.   Получить набор данных, готовый к применению
2.   Создать векторные представления узлов различными методами
3.   Провести классификацию узлов
4.   Сделать выводы о примененных методах



## install

In [ ]:
!pip install node2vec

In [ ]:
!pip install torch_geometric

In [ ]:
!pip install karateclub

## **2. Основная часть**

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from node2vec import Node2Vec
from karateclub.node_embedding.neighbourhood import GraRep, HOPE, DeepWalk
import random
from scipy.sparse import coo_matrix
from karateclub.node_embedding.attributed import SINE
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

## **2.1. Работа с данными**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
G0 = nx.read_edgelist('drive/MyDrive/users.edges', create_using = nx.DiGraph())
df0 = pd.read_csv('drive/MyDrive/users_neighborhood_anon.csv')
#df0.head()

Для классификации оставим только узлы, имеющие пометку.

In [ ]:
df = df0[df0.hate != 'other'].reset_index(drop=True)
df

In [ ]:
print(df.hate.value_counts())
df.hate.value_counts().plot(kind='bar')
plt.title('Распределение по классам')
plt.show()

Можно заметить, что классы являются несбалансированными, что может повлиять на дальнейшие результаты классификации, поэтому надо будет учесть это при выборе параметров моделей

In [ ]:
df.hate = pd.get_dummies(df, columns=['hate'], drop_first = True)['hate_normal']
df

In [ ]:
subgraph = G0.subgraph(map(str, df.user_id.to_list()))
print(subgraph)

Удалим петли, так как они не несут в себе смысловой нагрузки в рамках выбранных данных

In [ ]:
G = nx.relabel_nodes(subgraph, dict(zip(subgraph, map(int, subgraph.nodes()))))
G = nx.convert_node_labels_to_integers(G, ordering='sorted')
G.remove_edges_from(list(nx.selfloop_edges(G)))
print(G)
#nx.draw(G)

Далее представлена десятая часть сети, где красным помечены пользователи, разжигающие ненависть

In [ ]:
node_colors = list(map(lambda x: 'r' if x == 0 else 'b', df.hate))
nx.draw_networkx(G.subgraph(df.index.to_list()[::10]), node_color = node_colors[::10], with_labels = False)
plt.title('Граф (выборка)')
plt.show()

### **2.2. Векторные представления узлов**

Рассмотрим некоторые методы векторных представлений узлов, которые основаны на:

1) Матричном разложении

2) Случайных блужданиях

Для удобства при применении каждого из методов будем получать вектора одинаковой размерности

#### *2.2.1. Матричное разложение*

##### *Сингулярное разложение матрицы смежности узлов*

Самой простой способ получения векторных представлений узлов из графа с использованием матричной факторизации заключается в использовании метода Singular Value Decomposition (SVD), который уменьшает размерность матрицы смежности узлов посредством сингулярного разложения

In [ ]:
A = nx.to_numpy_array(G)
svd = TruncatedSVD(n_components=32)
vec_svd = svd.fit_transform(A)
vec_svd.shape

##### *GraRep (Learning Graph Representations with Global Structural Information)*

Метод GraRep основывается на вычислении матрицы переходов, элементы который являются вероятностями перехода из одного узла в другой, рассчитанными на основе количества общих соседей этих узлов

In [ ]:
model = GraRep(dimensions = 8, order = 4)
model.fit(G)

In [ ]:
vec_grarep = model.get_embedding()
vec_grarep.shape

##### *HOPE (Higher-Order Proximity Embeddings)*

Метод HOPE основан на идее, что узлы, которые имеют сходную структуру связей с другими узлами, должны иметь схожие векторные представления. Особенность алгоритма в том, что он учитывает связи выше второго порядка

In [ ]:
model = HOPE(dimensions = 32)
model.fit(G)

In [ ]:
vec_hope = model.get_embedding()
vec_hope.shape

#### *2.2.2. Случайные блуждания*

Методы, который вычисляют векторные представления узла на основе случайных блужданий в графе: DeepWalk и Node2Vec.

Node2Vec - это вариация DeepWalk, которая вводит смещенные
случайные блуждания. Случайными блужданиями управляют два параметра: p уменьшает
вероятность повторного посещения предыдущего узла, в то время
как q уменьшает вероятность перехода к узлам, которые
не были соседями исходного узла.

###### *DeepWalk*

In [ ]:
model = DeepWalk(dimensions = 32, walk_length=30, workers=4, walk_number = 50)
model.fit(G)

In [ ]:
vec_deepwalk = model.get_embedding()
vec_deepwalk.shape

##### *Node2Vec*

In [ ]:
node2vec = Node2Vec(G, dimensions=32, walk_length=50, num_walks=30, workers=4, p = 1.3, q = 0.7)
model = node2vec.fit(window=10, min_count=1)
vec_node2vec = np.array([model.wv[node] for node in G.nodes()])
vec_node2vec.shape

##### *SINE (Scalable Incomplete Network Embedding)*

Метод SINE позволяет использовать атрибуты узлов для построения векторов.

Процедура неявно факторизует совместную мощность матрицы смежности и матрицы признаков. Декомпозиция выполняется на основе усеченных случайных блужданий, и мощности матрицы смежности объединяются.

In [ ]:
df_features = df.iloc[:, 2:].select_dtypes(include='number')
features = df_features.to_numpy()

In [ ]:
X = coo_matrix(features)
model = SINE(dimensions = 32)
model.fit(G, X)

In [ ]:
vec_sine = model.get_embedding()
vec_sine.shape

### **2.3. Классификация узлов**

Чтобы получить более точные результаты сравнения, классификацию будем проводить разными методами:

1) Методы машинного обучения

2) Нейронная сеть

#### *2.3.1. Методы машинного обучения*

Разделим данные на обучающую и тестовую выборки, а также найдем веса классов

In [ ]:
train_mask, test_mask = train_test_split(df.index, test_size=0.2)
y = df.hate
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
cw = dict(zip(np.unique(y), class_weights))

##### *SVC (Support Vector Classifier)*

Работает путем нахождения гиперплоскости в многомерном пространстве, которая разделяет точки данных на разные классы.

In [ ]:
clf = svm.SVC(class_weight = cw)
clf.fit(vec_svd[train_mask], y[train_mask])

In [ ]:
svd_res = clf.predict(vec_svd[test_mask])
svc_res = [f1_score(y[test_mask], svd_res, average = 'macro')]
print(classification_report(svd_res, y[test_mask]))

In [ ]:
clf = svm.SVC(class_weight = cw)
clf.fit(vec_grarep[train_mask], y[train_mask])

In [ ]:
grarep_res = clf.predict(vec_grarep[test_mask])
svc_res.append(f1_score(y[test_mask], grarep_res, average = 'macro'))
print(classification_report(grarep_res, y[test_mask]))

In [ ]:
clf = svm.SVC(class_weight = cw)
clf.fit(vec_hope[train_mask], y[train_mask])

In [ ]:
hope_res = clf.predict(vec_hope[test_mask])
svc_res.append(f1_score(y[test_mask], hope_res, average = 'macro'))
print(classification_report(hope_res, y[test_mask]))

In [ ]:
clf = svm.SVC(class_weight = cw)
clf.fit(vec_deepwalk[train_mask], y[train_mask])

In [ ]:
deepwalk_res = clf.predict(vec_deepwalk[test_mask])
svc_res.append(f1_score(y[test_mask], deepwalk_res, average = 'macro'))
print(classification_report(deepwalk_res, y[test_mask]))

In [ ]:
clf = svm.SVC(class_weight = cw)
clf.fit(vec_node2vec[train_mask], y[train_mask])

In [ ]:
node2vec_res = clf.predict(vec_node2vec[test_mask])
svc_res.append(f1_score(y[test_mask], node2vec_res, average = 'macro'))
print(classification_report(node2vec_res, y[test_mask]))

In [ ]:
clf = svm.SVC(class_weight = cw)
clf.fit(vec_sine[train_mask], y[train_mask])

In [ ]:
sine_res = clf.predict(vec_sine[test_mask])
svc_res.append(f1_score(y[test_mask], sine_res, average = 'macro'))
print(classification_report(sine_res, y[test_mask]))

In [ ]:
pd.DataFrame(svc_res, columns=['f1_score'], index = ['svd','grarep','hope','deepwalk', 'node2vec', 'sine']).sort_values(by='f1_score', ascending = False)

Лучший результат по f1-score показал метод SINE, который при обучении векторов использует атрибуты узлов. Атрибуты узлов в свою очередь являются своеобразными характеристика поведения пользователей, поэтому метод может быть очень показателен при наличии таких данных.

Второе место у HOPE. Можно предположить, что это происходит из-за того, что HOPE, в отличие от остальных методов, рассматривает связи выше второго порядка, то есть углубляется в сеть и взаимосвязи между пользователями. В рамках взятых данных - социальная сеть с ретвитами - это может означать, что пользователи, разжигающие ненависть, имеют определенные не сразу заметные сходства.

На последних местах svd и Node2Vec. Первое не вызывает особых вопросов, в связи с поверхностностью построения векторов. Node2Vec, вероятно, показывает плохой результат из-за неправильно подобрабных параметров при обучении векторов.

##### *Random Forest*

Работает путем построения множества деревьев решений и объединения их предсказаний.


In [ ]:
clf = RandomForestClassifier(class_weight = cw)
clf.fit(vec_svd[train_mask], y[train_mask])

In [ ]:
svd_res = clf.predict(vec_svd[test_mask])
rf_res = [f1_score(y[test_mask], svd_res, average = 'macro')]
print(classification_report(svd_res, y[test_mask]))

In [ ]:
clf = RandomForestClassifier(class_weight = cw)
clf.fit(vec_grarep[train_mask], y[train_mask])

In [ ]:
grarep_res = clf.predict(vec_grarep[test_mask])
rf_res.append(f1_score(y[test_mask], grarep_res, average = 'macro'))
print(classification_report(grarep_res, y[test_mask]))

In [ ]:
clf = RandomForestClassifier(class_weight = cw)
clf.fit(vec_hope[train_mask], y[train_mask])

In [ ]:
hope_res = clf.predict(vec_hope[test_mask])
rf_res.append(f1_score(y[test_mask], hope_res, average = 'macro'))
print(classification_report(hope_res, y[test_mask]))

In [ ]:
clf = RandomForestClassifier(class_weight = cw)
clf.fit(vec_deepwalk[train_mask], y[train_mask])

In [ ]:
deepwalk_res = clf.predict(vec_deepwalk[test_mask])
rf_res.append(f1_score(y[test_mask], deepwalk_res, average = 'macro'))
print(classification_report(deepwalk_res, y[test_mask]))

In [ ]:
clf = RandomForestClassifier(class_weight = cw)
clf.fit(vec_node2vec[train_mask], y[train_mask])

In [ ]:
node2vec_res = clf.predict(vec_node2vec[test_mask])
rf_res.append(f1_score(y[test_mask], node2vec_res, average = 'macro'))
print(classification_report(node2vec_res, y[test_mask]))

In [ ]:
clf = RandomForestClassifier(class_weight = cw)
clf.fit(vec_sine[train_mask], y[train_mask])

In [ ]:
sine_res = clf.predict(vec_sine[test_mask])
rf_res.append(f1_score(y[test_mask], sine_res, average = 'macro'))
print(classification_report(sine_res, y[test_mask]))

In [ ]:
pd.DataFrame(rf_res, columns=['f1_score'], index = ['svd','grarep','hope','deepwalk', 'node2vec', 'sine']).sort_values(by='f1_score', ascending = False)

В случае модели Случайного леса лучшим стал снова метод SINE.

Можно еще выделить метод GraRep, который занял 2 место по f1-score, несмотря на относительно небольшую вычислительную мощность. Данный метод учитывает первостепенные и второстепенные связи, на основе которых, как показала практика, можно получить хорошие результаты классификации.

In [ ]:
r = np.arange(6)
width = 0.25

plt.bar(r, svc_res, color = 'b', width = width, edgecolor = 'black', label='SVC')
plt.bar(r + width, rf_res, color = 'g', width = width, edgecolor = 'black', label='Random Forest')

plt.xlabel("Методы")
plt.ylabel("f1-score")
plt.title("Результаты классификации")

plt.xticks(r + width/2,['svd','grarep','hope','deepwalk', 'node2vec', 'sine'])
plt.legend()
plt.show()

Обобщая результаты проведенной классификации двумя алгоритмами машинного обучения, можено сделать вывод, что для социальной сети лучше подходит SVC модель, которая находит оптимальную гиперплоскость для разделения классов. Так как данные имеют векторное представление, такой результат можно назвать закономерным

Интересно, что DeepWalk, который использует только случайные блуждания для построения векторных представлений, имеет большое различие между f1-score

#### *2.3.2. Нейронная сеть*

Далее попробуем классифицировать узлы при помощи графовой сверточной сети.

Сначала в качестве признаков узлов будем использовать атрибуты узлов, уменьшенные в размерности с помощью сингулярного разложения, а после - векторные представления узлов, полученные ранее

In [ ]:
class GCN(nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super().__init__()
        self.conv1 = GCNConv(n_input, n_hidden)
        self.conv2 = GCNConv(n_hidden, n_output)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

In [ ]:
n_epochs = 301
n_input = 32
n_hidden = 128
n_out = 2

In [ ]:
def visualize():
  model.eval()
  with torch.no_grad():
      predictions = model(data)

  for cl in labels.unique():
      plt.scatter(predictions[cl == labels, 0], predictions[cl == labels, 1], label=str(cl.item()), linewidths = 5)
  plt.legend()
  plt.show()

In [ ]:
model = GCN(n_input, n_hidden, n_out)

svd = TruncatedSVD(n_components=32)
features32 = svd.fit_transform(df_features.fillna(df.mean(numeric_only=True)).to_numpy())

labels = torch.tensor(y).to(torch.int64)
edges = torch.tensor(list(G.edges)).t().contiguous().long()
data = Data(x = torch.from_numpy(features32).to(torch.float32), edge_index = edges)

plt.title('Предсказания необученной модели')
visualize()

In [ ]:
model = GCN(n_input, n_hidden, n_out)

optimizer = optim.Adam(model.parameters(), lr=.01)
criterion = nn.CrossEntropyLoss()

for epoch in range(n_epochs):
    logits = model(data)
    loss = criterion(logits[train_mask], labels[train_mask])

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    with torch.no_grad():
        predictions = logits.argmax(dim=1)
        train_acc = (predictions[train_mask] == labels[train_mask]).float().mean()
        test_acc = (predictions[test_mask] == labels[test_mask]).float().mean()

    if not epoch % 20:
        print(f'In epoch {epoch}, train acc: {train_acc:.3f}, test acc: {test_acc:.3f}')

gcn_res = [f1_score(predictions[test_mask], labels[test_mask], average = 'macro')]

In [ ]:
plt.title('Предсказания модели, обученной на атрибутах узлов')
visualize()

In [ ]:
model = GCN(n_input, n_hidden, n_out)

optimizer = optim.Adam(model.parameters(), lr=.01)
criterion = nn.CrossEntropyLoss()

data = Data(x = torch.from_numpy(vec_svd).to(torch.float32), edge_index = edges)

for epoch in range(n_epochs):
    logits = model(data)
    loss = criterion(logits[train_mask], labels[train_mask])

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    with torch.no_grad():
        predictions = logits.argmax(dim=1)
        train_acc = (predictions[train_mask] == labels[train_mask]).float().mean()
        test_acc = (predictions[test_mask] == labels[test_mask]).float().mean()

    if not epoch % 40:
        print(f'In epoch {epoch}, train acc: {train_acc:.3f}, test acc: {test_acc:.3f}')

gcn_res.append(f1_score(predictions[test_mask], labels[test_mask], average = 'macro'))

In [ ]:
plt.title('Предсказания модели, обученной на векторах svd')
visualize()

In [ ]:
model = GCN(n_input, n_hidden, n_out)

optimizer = optim.Adam(model.parameters(), lr=.01)
criterion = nn.CrossEntropyLoss()

data = Data(x = torch.from_numpy(vec_grarep).to(torch.float32), edge_index = edges)

for epoch in range(n_epochs):
    logits = model(data)
    loss = criterion(logits[train_mask], labels[train_mask])

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    with torch.no_grad():
        predictions = logits.argmax(dim=1)
        train_acc = (predictions[train_mask] == labels[train_mask]).float().mean()
        test_acc = (predictions[test_mask] == labels[test_mask]).float().mean()

    if not epoch % 40:
        print(f'In epoch {epoch}, train acc: {train_acc:.3f}, test acc: {test_acc:.3f}')

gcn_res.append(f1_score(predictions[test_mask], labels[test_mask], average = 'macro'))

In [ ]:
plt.title('Предсказания модели, обученной на векторах GraRep')
visualize()

In [ ]:
model = GCN(n_input, n_hidden, n_out)

optimizer = optim.Adam(model.parameters(), lr=.01)
criterion = nn.CrossEntropyLoss()

data = Data(x = torch.from_numpy(vec_hope).to(torch.float32), edge_index = edges)

for epoch in range(n_epochs):
    logits = model(data)
    loss = criterion(logits[train_mask], labels[train_mask])

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    with torch.no_grad():
        predictions = logits.argmax(dim=1)
        train_acc = (predictions[train_mask] == labels[train_mask]).float().mean()
        test_acc = (predictions[test_mask] == labels[test_mask]).float().mean()

    if not epoch % 40:
        print(f'In epoch {epoch}, train acc: {train_acc:.3f}, test acc: {test_acc:.3f}')

gcn_res.append(f1_score(predictions[test_mask], labels[test_mask], average = 'macro'))

In [ ]:
plt.title('Предсказания модели, обученной на векторах HOPE')
visualize()

In [ ]:
model = GCN(n_input, n_hidden, n_out)

optimizer = optim.Adam(model.parameters(), lr=.01)
criterion = nn.CrossEntropyLoss()

data = Data(x = torch.from_numpy(vec_deepwalk).to(torch.float32), edge_index = edges)

for epoch in range(n_epochs):
    logits = model(data)
    loss = criterion(logits[train_mask], labels[train_mask])

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    with torch.no_grad():
        predictions = logits.argmax(dim=1)
        train_acc = (predictions[train_mask] == labels[train_mask]).float().mean()
        test_acc = (predictions[test_mask] == labels[test_mask]).float().mean()

    if not epoch % 40:
        print(f'In epoch {epoch}, train acc: {train_acc:.3f}, test acc: {test_acc:.3f}')

gcn_res.append(f1_score(predictions[test_mask], labels[test_mask], average = 'macro'))

In [ ]:
plt.title('Предсказания модели, обученной на векторах DeepWalk')
visualize()

In [ ]:
model = GCN(n_input, n_hidden, n_out)

optimizer = optim.Adam(model.parameters(), lr=.01)
criterion = nn.CrossEntropyLoss()

data = Data(x = torch.from_numpy(vec_node2vec).to(torch.float32), edge_index = edges)

for epoch in range(n_epochs):
    logits = model(data)
    loss = criterion(logits[train_mask], labels[train_mask])

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    with torch.no_grad():
        predictions = logits.argmax(dim=1)
        train_acc = (predictions[train_mask] == labels[train_mask]).float().mean()
        test_acc = (predictions[test_mask] == labels[test_mask]).float().mean()

    if not epoch % 40:
        print(f'In epoch {epoch}, train acc: {train_acc:.3f}, test acc: {test_acc:.3f}')

gcn_res.append(f1_score(predictions[test_mask], labels[test_mask], average = 'macro'))

In [ ]:
plt.title('Предсказания модели, обученной на векторах Node2Vec')
visualize()

In [ ]:
model = GCN(n_input, n_hidden, n_out)

optimizer = optim.Adam(model.parameters(), lr=.01)
criterion = nn.CrossEntropyLoss()

data = Data(x = torch.from_numpy(vec_sine).to(torch.float32), edge_index = edges)

for epoch in range(n_epochs):
    logits = model(data)
    loss = criterion(logits[train_mask], labels[train_mask])

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    with torch.no_grad():
        predictions = logits.argmax(dim=1)
        train_acc = (predictions[train_mask] == labels[train_mask]).float().mean()
        test_acc = (predictions[test_mask] == labels[test_mask]).float().mean()

    if not epoch % 40:
        print(f'In epoch {epoch}, train acc: {train_acc:.3f}, test acc: {test_acc:.3f}')

gcn_res.append(f1_score(predictions[test_mask], labels[test_mask], average = 'macro'))

In [ ]:
plt.title('Предсказания модели, обученной на векторах SINE')
visualize()

In [ ]:
r = np.arange(7)

plt.bar(r, gcn_res, color = 'b', edgecolor = 'black')

plt.xlabel("Методы")
plt.ylabel("f1-score")
plt.title("Результаты классификации с помощью GCN")

plt.xticks(r,['features', 'svd','grarep','hope','deepwalk', 'node2vec', 'sine'])
plt.show()

In [ ]:
pd.DataFrame(gcn_res, columns=['f1_score'], index = ['features','svd','grarep','hope','deepwalk', 'node2vec', 'sine']).sort_values(by='f1_score', ascending = False)

При одинаковых параметров обучения, лучше всего себя показали те же методы, что и раньше. При этом обучение только на атрибутах узлов дало довольно плохой результат, в отличие от векторных представлений, полученных в ходе работы.

## **3. Заключение**

По итогам работы мы получили результаты классификации узлов сети несколькими способами. Можно сказать, что методы, глубоко рассматривающие взаимосвязи между узлами, лучше справляются с задачей классификации. Конечно, данные выводы распространяются только на выбранный набор данных и, вероятнее всего, могут быть распространены только на данные со схожей структурой.

В рамках задачи классификации пользователей, разжигающих ненависть, хорошо себя показал метод SINE, учитывающий и отношения между объектами сети, и атрибуты узлов. А также методы HOPE и GraRep, первый их которых включает в рассмотрение связи выше второго порядка, а второй - первого и второго.

Говоря о алгоритмах классификации, поскольку для обучения не подбирались специальные параметры, сложно сделать вывод о том, какой является наилучшим в широком смысле. Однако в ходе проведенного эксперимента при прочих равных условиях наибольшие значения получились при использовании SVC.

## **Список используемых источников:**

1.   Данные - https://www.kaggle.com/datasets/manoelribeiro/hateful-users-on-twitter
2.   Karate Club: An API Oriented Open-source Python Framework for Unsupervised Learning on Graphs (CIKM 2020) - https://github.com/benedekrozemberczki/karateclub
3.   Под капотом графовых сетей - https://habr.com/ru/articles/794558/
4.   Characterizing and Detecting Hateful Users on Twitter - https://www.researchgate.net/publication/365061339_Characterizing_and_Detecting_Hateful_Users_on_Twitter

